# Pitcher summary

The point of this notebook is to start collecting the summary data on pitchers. This will then be used for future analysis/wrangling.

In [1]:
import pybaseball as pyb

import pandas as pd
import numpy as np
import requests
import time

from bs4 import BeautifulSoup

## Pitcher summary

A summary of all pitchers. I manually downloaded the data from FanGraphs season-by-season, and then just joined them together to get `fg_pitchers.csv`. Technically `pybaseball` can do a similar thing, but when using it it only pulled a subset of the pitchers. So this seems more reliable.

In [131]:
fg_pitching_df = pd.read_csv('../data/fg_pitchers.csv')

In [132]:
fg_pitching_df = fg_pitching_df.sort_values('year')

In [133]:
fg_pitching_df.head()

,Name,Team,W,L,SV,G,GS,IP,K/9,BB/9,...,LOB%,GB%,HR/FB,EV,ERA,FIP,xFIP,WAR,playerid,year
0,John Frascatore,Blue Jays,2,4,0,60,0,73.0,3.70,4.07,...,70.8%,NaN,NaN,NaN,5.42,6.45,NaN,-1.1,1004304,2000
400,Mike Lincoln,Twins,0,3,0,8,4,20.2,6.53,5.66,...,70.3%,NaN,NaN,NaN,10.89,10.15,NaN,-0.8,1457,2000
401,Alan Mills,- - -,4,1,2,41,0,49.1,6.57,6.39,...,79.6%,NaN,NaN,NaN,5.29,6.30,NaN,-0.7,1008949,2000
402,Rick Aguilera,Cubs,1,2,29,54,0,47.2,7.17,3.40,...,76.5%,NaN,NaN,NaN,4.91,5.92,NaN,-0.7,1000086,2000
403,Chris Fussell,Royals,5,3,0,20,9,70.0,5.91,5.66,...,72.3%,NaN,NaN,NaN,6.30,7.13,NaN,-0.7,1004410,2000


Get their first and last seasons, starting and ending age, and number of games played. Also record all the teams they played for, and how many this is.

In [134]:
pitchers_summary_df = fg_pitching_df.groupby(['Name', 'playerid']).agg({'year': [min, max], 
                                                              'G': sum, 'GS': sum})
pitchers_summary_df['teams'] = fg_pitching_df.groupby(['Name', 'playerid'])['Team'].unique()
pitchers_summary_df['num_teams'] = fg_pitching_df.groupby(['Name', 'playerid'])['Team'].nunique()
pitchers_summary_df.head(10)

year          G   GS  \
                        min   max  sum  sum   
Name         playerid                         
A.J. Achter  11387     2014  2016   45    0   
A.J. Burnett 512       2000  2015  428  423   
A.J. Cole    11467     2015  2019   79   19   
A.J. Griffin 11132     2012  2017   88   85   
A.J. Minter  18655     2017  2019  117    0   
A.J. Morris  9919      2016  2016    7    0   
A.J. Murray  3422      2007  2008   16    4   
A.J. Puk     19343     2019  2019   10    0   
A.J. Reed    16246     2019  2019    1    0   
A.J. Schugel 11432     2015  2017   73    0   

                                                                  teams  \
                                                                          
Name         playerid                                                     
A.J. Achter  11387                                      [Twins, Angels]   
A.J. Burnett 512       [Marlins, Blue Jays, Yankees, Pirates, Phillies]   
A.J. Cole    11467                          [Nationals, - - -, Indians]   
A.J. Griffin 11132                                 [Athletics, Rangers]   
A.J. Minter  18655                                             [Braves]   
A.J. Morris  9919                                                [Reds]   
A.J. Murray  3422                                             [Rangers]   
A.J. Puk     19343                                          [Athletics]   
A.J. Reed    16246                                          [White Sox]   
A.J. Schugel 11432                              [Diamondbacks, Pirates]   

                      num_teams  
                                 
Name         playerid            
A.J. Achter  11387            2  
A.J. Burnett 512              5  
A.J. Cole    11467            3  
A.J. Griffin 11132            2  
A.J. Minter  18655            1  
A.J. Morris  9919             1  
A.J. Murray  3422             1  
A.J. Puk     19343            1  
A.J. Reed    16246            1  
A.J. Schugel 11432            2

In [135]:
pitchers_summary_df.columns = ['first_season', 'last_season', 'games_played', 'games_started', 'teams', 'num_teams']

In [136]:
pitchers_summary_df = pitchers_summary_df.reset_index()

In [137]:
# Store it as a float because missing values can't exist with ints (important when joining later)
pitchers_summary_df['playerid'] = pitchers_summary_df['playerid'].astype(float)

In [138]:
pitchers_summary_df.head()

,Name,playerid,first_season,last_season,games_played,games_started,teams,num_teams
0,A.J. Achter,11387.0,2014,2016,45,0,"[Twins, Angels]",2
1,A.J. Burnett,512.0,2000,2015,428,423,"[Marlins, Blue Jays, Yankees, Pirates, Phillies]",5
2,A.J. Cole,11467.0,2015,2019,79,19,"[Nationals, - - -, Indians]",3
3,A.J. Griffin,11132.0,2012,2017,88,85,"[Athletics, Rangers]",2
4,A.J. Minter,18655.0,2017,2019,117,0,[Braves],1


In [141]:
pitchers_summary_df['playerid'] = pitchers_summary_df['playerid'].astype(int)

Search by name. Note that **this isn't necessary in this case**, because fangraphs has their ID, so we can just do a reverse lookup, which is much faster. Saving it though in case this is useful in other situations.

In [ ]:
# pitcher_keys = []
# ten_pct_inc = int(pitchers_summary_df['Name'].nunique() / 10)

# for i, name in enumerate(pitchers_summary_df['Name'].unique()):
#     # Try and get their first and last name to search for. If this splits into more than
#     # just two parts, record it and move on
#     try:
#         first, last = name.split(' ', 1)
#         if '.' in first:
#             first = first.replace('.', '. ')
#             first = first.rstrip(' ')
#     except Exception as e:
#         row = [name] + [None]*4
#         pitcher_keys.append(row)
#         continue
        
#     # If you get a first and last name, look them up. If this returns more than one player,
#     # record it and move on. If not, get their data and 
#     pitcher_data = pyb.playerid_lookup(last, first)
#     if pitcher_data.shape[0] > 1:
#         row = [name] + [None]*4
#         pitcher_keys.append(row)
#         continue
#     else:
#         try:
#             row = [name] + list(pitcher_data[['key_mlbam', 'key_retro', 'key_bbref', 'key_fangraphs']].values[0])
#         except Exception as e:
#             row = [name] + [None]*4
#         pitcher_keys.append(row)
        
#     # Sleep for one second to avoid rate limiting
#     time.sleep(1)
    
#     if i % ten_pct_inc == 0:
#         print(f'{10*i/ten_pct_inc}% complete')

In [139]:
player_keys = pyb.playerid_reverse_lookup(pitchers_summary_df['playerid'], key_type='fangraphs')

Gathering player lookup table. This may take a moment.


In [140]:
player_keys.head()

,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
0,aardsma,david,430911,aardd001,aardsda01,1902,2004.0,2015.0
1,abad,fernando,472551,abadf001,abadfe01,4994,2010.0,2019.0
2,abbott,paul,110015,abbop001,abbotpa01,1061,1990.0,2004.0
3,abreu,bryan,650556,abreb002,abreubr01,16609,2019.0,2020.0
4,abreu,juan,444874,abrej002,abreuju01,6306,2011.0,2011.0


In [142]:
player_keys = player_keys[['key_mlbam', 'key_retro', 'key_bbref', 'key_fangraphs']]
player_keys.head()

,key_mlbam,key_retro,key_bbref,key_fangraphs
0,430911,aardd001,aardsda01,1902
1,472551,abadf001,abadfe01,4994
2,110015,abbop001,abbotpa01,1061
3,650556,abreb002,abreubr01,16609
4,444874,abrej002,abreuju01,6306


In [143]:
pitchers_summary_df = pitchers_summary_df.merge(player_keys, left_on='playerid', right_on='key_fangraphs')

In [144]:
pitchers_summary_df.head()

,Name,playerid,first_season,last_season,games_played,games_started,teams,num_teams,key_mlbam,key_retro,key_bbref,key_fangraphs
0,A.J. Achter,11387,2014,2016,45,0,"[Twins, Angels]",2,592091,achta001,achteaj01,11387
1,A.J. Burnett,512,2000,2015,428,423,"[Marlins, Blue Jays, Yankees, Pirates, Phillies]",5,150359,burna001,burnea.01,512
2,A.J. Cole,11467,2015,2019,79,19,"[Nationals, - - -, Indians]",3,595918,colea002,coleaj01,11467
3,A.J. Griffin,11132,2012,2017,88,85,"[Athletics, Rangers]",2,456167,grifa002,griffaj01,11132
4,A.J. Minter,18655,2017,2019,117,0,[Braves],1,621345,minta001,minteaj01,18655


In [145]:
pitchers_summary_df = pitchers_summary_df.drop('playerid', axis='columns')

In [146]:
pitchers_summary_df.isna().sum()

Name             0
first_season     0
last_season      0
games_played     0
games_started    0
teams            0
num_teams        0
key_mlbam        0
key_retro        0
key_bbref        0
key_fangraphs    0
dtype: int64

In [147]:
pitchers_summary_df.shape[0]

3308

In [148]:
pitchers_summary_df.to_csv('../data/pitchers_summary.csv', index=False)

## Fetch pitchers game-by-game data from BR

pybaseball doesn't seem to give you access to game-by-game stats for pitchers, which I need. So taking their code and modifying it to pull directly from BR. Note that this _does_ include ERA, but _doesn't_ include WHIP.

In [3]:
pitchers_summary_df = pd.read_csv('../data/pitchers_summary.csv')

In [4]:
pitchers_summary_df.head()

,Name,first_season,last_season,start_age,end_age,games_played,teams,num_teams,key_mlbam,key_retro,key_bbref,key_fangraphs
0,A.J. Burnett,2001,2015,24,38,370,['Blue Jays' 'Marlins' 'Pirates' 'Yankees' 'Ph...,5,150359.0,burna001,burnea.01,512.0
1,A.J. Griffin,2013,2013,25,25,32,['Athletics'],1,456167.0,grifa002,griffaj01,11132.0
2,Aaron Cook,2006,2008,27,29,89,['Rockies'],1,346871.0,cooka002,cookaa01,1571.0
3,Aaron Harang,2005,2015,27,37,279,['Reds' 'Braves' 'Dodgers' 'Padres' 'Phillies'],5,421685.0,haraa001,haranaa01,1451.0
4,Aaron Nola,2017,2019,24,26,94,['Phillies'],1,605400.0,nolaa001,nolaaa01,16149.0


Get pitcher game-level data from Baseball Reference.

In [222]:
def pitcher_bref(br_id, season):
    """
    Get season-level Pitching Statistics for Specific Team (from Baseball-Reference)
    ARGUMENTS:
    br_id : str : The BR unique identifier. You can get this from playerid_lookup in the key_bberf columns
    end_season : int : season you want data for (data is returned on a game-by-game basis)
    """

    url = f"https://www.baseball-reference.com/players/gl.fcgi?id={br_id}&t=p&year={season}"

    data = []
    headings = None
    stats_url = url
    response = requests.get(stats_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    table = soup.find_all('table', {'id': 'pitching_gamelogs'})
    if len(table) > 0:
        table = table[0]
    else:
        return None

    if headings is None:
        headings = [row.text.strip() for row in table.find_all('th')]
        # Even within BR, it seems like different years (or perhaps players?) have
        # different numbers of columns (i.e. different stats being calculated). Annoyingly,
        # finding all "th" includes a bunch of junk column headers. You can identify where 
        # the junk ones start, because you just start getting integers (e.g. "5", "6", etc.).
        # So my work around here is to look for the first integer, and then just filter out
        # everything after that point.
        for i, h in enumerate(headings):
            try:
                int(h)
                headings = headings[1:i]
                break
            except Exception as e:
                pass

    rows = table.find_all('tr')
    # Skip the last row, as this is a footer with only yearly summary data
    for row in rows[:-1]:
        cols = row.find_all('td')
        # Some rows are just basically a few long td's with text, explaining things
        # like trades. This is a really crude way to filter those out.
        if len(cols) > 40:
            cols = [ele.text.strip() for ele in cols]
            cols = [col.replace('*', '').replace('#', '') for col in cols]  # Removes '*' and '#' from some names
            cols = [col for col in cols if 'Totals' not in col and 'NL teams' not in col and 'AL teams' not in col]  # Removes Team Totals and other rows
            cols.insert(2, int(season))
            data.append([ele for ele in cols[0:]])

    headings.insert(2, "Year")
    
    data = pd.DataFrame(data=data, columns=headings) # [:-5]  # -5 to remove Team Totals and other rows (didn't work in multi-year queries)
    data.columns = [x if x != '' else 'at' for x in data.columns]
    data = data.dropna()  # Removes Row of All Nones
    data.reset_index(drop=True, inplace=True)  # Fixes index issue (Index was named 'W" for some reason)
    
    return data

Clean up the data and calculate WHIP.

In [220]:
def get_pitcher_game_stats(br_id, year):
    pitcher_df = pitcher_bref(br_id, year)
    if pitcher_df is None:
        return None
    pitcher_df['Year'] = pitcher_df['Year'].astype(int).astype(str)
    pitcher_df['Date'] = pitcher_df[['Date', 'Year']].agg(' '.join, axis=1)
    
    # Note double-headers and remove the (#) marker from the date so that Pandas can parse it
    patt = r'\((\d+)\)'
    pitcher_df['Double_Header'] = pitcher_df['Date'].str.extract(patt, expand=False)
    pitcher_df['Date'] = pitcher_df['Date'].str.replace(patt, '')
    
    # Weird date formatting
    pitcher_df['Date'] = pitcher_df['Date'].str.replace('susp', '')
    
    pitcher_df = pitcher_df.drop('Year', axis='columns')
    
    home_team = []
    for i in range(pitcher_df.shape[0]):
        if pitcher_df.loc[i, 'at'] == '@':
            home_team.append(pitcher_df.loc[i, 'Opp'])
        else:
            home_team.append(pitcher_df.loc[i, 'Tm'])
            
    pitcher_df['Home_Tm'] = home_team
    pitcher_df = pitcher_df.drop('at', axis='columns')
    
    pitcher_df['WHIP'] = (pitcher_df['BB'].astype(int) + pitcher_df['H'].astype(int)) / pitcher_df['IP'].astype(float)
    
    pitcher_df[['Result', 'Final_Score']] = pitcher_df['Rslt'].str.split(',', 1, expand=True)
    pitcher_df[['Tm_Score', 'Opp_Score']] = pitcher_df['Final_Score'].str.split('-', 1, expand=True)
    pitcher_df = pitcher_df.drop(['Rslt', 'Final_Score'], axis='columns')
    return pitcher_df

In [215]:
acook = get_pitcher_game_stats('cookaa01', 2002)

['Gcar', 'Gtm', 'Date', 'Tm', '', 'Opp', 'Rslt', 'Inngs', 'Dec', 'DR', 'IP', 'H', 'R', 'ER', 'BB', 'SO', 'HR', 'HBP', 'ERA', 'BF', 'Pit', 'Str', 'StL', 'StS', 'GB', 'FB', 'LD', 'PU', 'Unk', 'GSc', 'IR', 'IS', 'SB', 'CS', 'PO', 'AB', '2B', '3B', 'IBB', 'GDP', 'SF', 'ROE', 'aLI', 'WPA', 'acLI', 'cWPA', 'RE24', 'Entered', 'Exited']


In [216]:
acook

,Gcar,Gtm,Date,Tm,Opp,Inngs,Dec,DR,IP,H,...,cWPA,RE24,Entered,Exited,Double_Header,Home_Tm,WHIP,Result,Tm_Score,Opp_Score
0,1,116,Aug 10 2002,COL,CHC,6-7,,99,2.0,2,...,0.00%,0.14,6t --- 0 out d13,7t 3 out d14,NaN,COL,1.000000,L,1,15
1,2,121,Aug 16 2002,COL,ATL,8-GF(8),,5,1.0,0,...,0.00%,0.51,8b --- 0 out d3,8b end d 3,NaN,ATL,0.000000,L,1,4
2,3,125,Aug 20 2002,COL,MON,6-7,,3,1.2,3,...,0.00%,-2.12,6t --- 0 out a5,7t 1-- 2 out a2,NaN,COL,3.333333,W,8,6
3,4,127,Aug 22 2002,COL,MON,6-6,H(1),1,1.0,0,...,0.00%,0.57,6t --- 0 out a3,6t 3 out a3,NaN,COL,1.000000,W,14,6
4,5,131,Aug 26 2002,COL,SFG,GS-7,,3,6.0,9,...,0.00%,-0.60,1t start tie,7t 12- 0 out d2,NaN,COL,2.166667,L,3,4
5,6,136,Aug 31 2002,COL,SDP,GS-7,L(0-1),4,7.0,9,...,0.00%,0.12,1b start tie,7b 3 out d3,NaN,SDP,1.285714,L,0,3
6,7,141,Sep 6 2002,COL,SDP,GS-7,W(1-1),5,7.0,5,...,0.00%,2.99,1t start tie,7t 3 out a6,NaN,COL,1.000000,W,7,3
7,8,146,Sep 11 2002,COL,HOU,GS-6,W(2-1),4,6.0,7,...,0.00%,0.16,1b start tie,6b 3 out a3,NaN,HOU,1.333333,W,8,6
8,9,151,Sep 17 2002,COL,STL,GS-4,,5,4.0,6,...,0.00%,-1.72,1t start tie,4t 3 out d3,NaN,COL,2.500000,L,4,11


Get this data for all pitchers and save it.

In [ ]:
successfully_processed_pitchers = []
failed_processing_pitchers = []
num_pitchers = pitchers_summary_df['key_bbref'].nunique()

for i, key in enumerate(pitchers_summary_df['key_bbref'].unique()):
    pitcher_yearly_df = None
    try:
        for y in range(2000, 2020):
            year_df = get_pitcher_game_stats(key, y)
            if year_df is not None:
                if pitcher_yearly_df is None:
                    pitcher_yearly_df = year_df
                else:
                    pitcher_yearly_df = pd.concat([pitcher_yearly_df, year_df])
        pitcher_yearly_df['Date'] = pd.to_datetime(pitcher_yearly_df['Date'])
        pitcher_yearly_df = pitcher_yearly_df.sort_values('Date')
        pitcher_yearly_df.to_csv(f'../data/pitchers_games/{key}.csv', index=False)
        successfully_processed_pitchers.append(key)
    except Exception as e:
        failed_processing_pitchers.append(key)
    
    if i % 50 == 0 and i > 0:
        num_success = len(successfully_processed_pitchers)
        num_fails = len(failed_processing_pitchers)
        print(f'{100*(i+1) / num_pitchers:.2f}% processed')
        print(f'{num_success} ({100*num_success / (num_success + num_fails):.2f}%) successful')
        print('='*40)
    time.sleep(1)

1.54% processed
51 (100.00%) successful
3.05% processed
101 (100.00%) successful
4.56% processed
151 (100.00%) successful
6.08% processed
201 (100.00%) successful
7.59% processed
251 (100.00%) successful


Examples of this data can be seen [here](https://www.baseball-reference.com/players/gl.fcgi?id=cookaa01&t=p&year=2002).

Descriptions of columns:
- Gcar -- Career Game Number for Player
- Gtm -- Season Game Number for Team. Number in parentheses indicates number of team games the player did not play in from one appearance to next.
- Date -- A number in parentheses indicates which game of a doubleheader.
- Rslt -- Game Result for Team. W - Win, L - Loss, T - Tie (for a suspended game)
- Inngs -- Innings Played by Player
    - CG - Complete Game started and finished
    - GS-# - Game Started to what inning
    - #-GF, Inning entered to end of game
    - #-# - Inning Entered to Inning Left
    - (#) Game did not go 9 innings (only shown when player finished the game).
    - For pitchers, an SHO means they shutout the opposition. A zero for the innings means the innings played is unknown.
- Dec -- Decision, Save, or Hold
    - W - Win (pitcher record after game)
    - L - Loss (pitcher record after game)
    - BW - Blown Save and Win (pitcher record after game)
    - BL - Blown Save and Loss (pitcher record after game)
    - S - Save (pitcher saves thus far)
    - BSv - Blown Save (pitcher blown saves thus far)
    - H - Hold (pitcher holds thus far)
- DR -- Days Rest. Number or days since their previous appearance. 99 if start of season or 99 or more days (may include demotions). -1 if pitching both games of double-header.
- IP -- Innings Pitched
- H -- Hits/Hits Allowed
- R -- Runs Scored/Allowed
- ER -- Earned Runs Allowed
- BB -- Bases on Balls/Walks
- SO -- Strikeouts
- HR -- Home Runs Hit/Allowed
- HBP -- Times Hit by a Pitch.
- ERA -- 9 * ER / IP. For recent years, leaders need 1 IP per team game played.
- BF -- Batters Faced
- Pit -- Number of pitches in the PA.
- Str -- Strikes. Includes both pitches in the zone and those swung at out of the zone.
- StL -- Strikes Looking. Strikes called by the umpire.
- StS -- Strikes Swinging. Strikes due to a swing and a miss.
- GB -- Ground Balls. Includes bunts and all other ground balls.
- FB -- Fly Balls. Includes Fly Balls, Line Drives, and Pop-Ups.
- LD -- Line Drives. These are double-counted in Fly Balls as well.
- PU -- Pop Ups. Generally, high fly balls that land within the infield circle. These are double-counted in Fly Balls as well.
- Unk -- Unknown batted ball type. A ball in play for which we don’t know the type.
- GSc -- Game Score. Developed by Bill James
    1. Start with 50 points.
    2. Add 1 point for each out recorded, so 3 points for every complete inning pitched.
    3. Add 2 points for each inning completed after the 4th.
    4. Add 1 point for each strikeout.
    5. Subtract 2 points for each hit allowed.
    6. Subtract 4 points for each earned run allowed.
    7. Subtract 2 points for each unearned run allowed.
    8. Subtract 1 point for each walk.
- IR -- Inherited Runners. Number of runners on base when pitcher entered the game.
- IS -- Inherited Score. Number or percentage of runners on base when pitcher entered the game who subsequently scored. These runners show up in the previous pitcher’s ERA.
- SB -- Stolen Bases
- CS -- Caught Stealing
- PO -- Pickoffs. Runner picked off a base. May include cases they were safe on an error. Also includes Pickoff Caught Stealing plays.
- AB -- At Bats
- 2B -- Doubles Hit/Allowed
- 3B -- Triples Hit/Allowed
- IBB -- Intentional Bases on Balls
- GDP -- Double Plays Grounded Into. Only includes standard 6-4-3, 4-3, etc. double plays. For gamelogs only in seasons we have play-by-play, we include triple plays as well. All official seasonal totals do not include GITP's.
- SF -- Sacrifice Flies
- ROE -- Reached On Error. Times a batter reached due to an error. DOES NOT include a fielder’s choice where no out was recorded.
- aLI -- Average Leverage Index. The average pressure the pitcher or batter saw in this game or season. 1.0 is average pressure, below 1.0 is low pressure and above 1.0 is high pressure.
- WPA -- Win Probability Added by Pitcher. Given average teams, this is the change in probability. A change of +/- 1 would indicate one win added or lost.
- acLI -- Average Championship Leverage Index. The average pressure the pitcher or batter saw in this game or season. 1.0 is average pressure, below 1.0 is low pressure and above 1.0 is high pressure.
- cWPA -- Championship Win Probability Added by Pitcher. Given average teams, this is the change in probability, displayed in percentage points. A change of +/- 100% would indicate one world series win added or lost.
- RE24 -- Base-Out Runs Saved. Given the bases occupied/out situation, how many runs did the pitcher save in the resulting play. Compared to average, so 0 is average, and above 0 is better than average
- Entered -- The situation when pitcher entered game. 
    - Inning top or bottom: 8b (bottom of 8th) 
    - bases occupied or start of inning: ’---’ (bases empty) 
    - score from pitching team’s perspective 
        - ahead/down and runs or tie: a4 (ahead by 4 runs) 
- Exited -- The situation when pitcher exited game
    - Inning top or bottom: 4t (top of 4th)
    - bases occupied, 3 outs, or end of game: ’123’ (bases loaded)
    - score from pitching team’s perspective
        - ahead/down and runs or tie: d2 (down by 2 runs)